In [1]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance

# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
print(os.getcwd())

C:\Users\vgu\AppData\Local\miniconda3\envs\bertopic-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\vgu\Documents\Topic Modeling


In [42]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

In [43]:
# Training-Datensatz
training_set = pd.read_csv("training_set.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame
# training_set = training_set.sample(n=500, random_state=42)  # Zieht eine Zufallsstichprobe von 500 Zeilen aus dem DataFrame mit festgelegtem Seed für Reproduzierbarkeit
training_set = training_set.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
training_set['titel_kursbesch'] = training_set['titel'] + ' ' + training_set['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = training_set['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

In [44]:
# Test-Datensatz
test_data = pd.read_csv("test_set.csv", sep=";")
test_set = test_data["Volltext"]  # Texte der Test-Daten
ground_truth = test_data["Keywords"]  # Spalte im CSV mit manuell erstellten Begriffen, welche man als korrekt erachtet (i.d.R. einfach wichtige Wörter aus dem Text rauskopieren)

## Stopwords

In [45]:
irrelevant_terms = [
    "vl",
    "übung",
    "übungen",
    "seminar",
    "arbeitsgruppenseminar",
    "oberseminar",
    "proseminar",
    "blockveranstaltung",
    "vorlesung",
    "kolloquium",
    "theoriekolloquium",
    "einführung",
    "tutorium",
    "ue",
    "vereinbarung",
    "projekt",
    "praktikum",
    "masterprojekt",
    "wiederholerklausur",
    "fortgeschrittenenpraktikum",
    "hauptseminar",
    "fachpraktikum",
    "ergänzungsvorlesung",
    "forschungspraktikum",
    "begleitseminar",
    "abschlussarbeiten",
    "unterrichtspraktikum",
    "masterseminar",
    "proseminare",
    "praxisseminar",
    "praxissemester",
    "schulpraxis",
    "ringpraktikum",
    "basispraktikum",
    "praxistage",
    "industriepraktikum",
    "vorkurs",
    "projektseminar",
    "juniorprofessur",
    "masterarbeiten",
    "forschungsseminar",
    "modulbeschreibung",
    "veranstaltung",
    "kommentare",
    "raum",
    "uhrzeit",
    "vereinbarung",
    "vorlesung",



    # "Informatik","gleichnamigen", "übungsaufgaben", "vorlesungsstoff vertieft", "vorlesungsstoff", "grundlagen programmierung", "schriftliche übungsaufgaben", "vertieft schriftliche übungsaufgaben", "vorlesungsstoff vertieft schriftliche", "vertieft schriftliche", "lehre",
    # "siehe", "klausur", "inf misc", "sitzungen", "misc", "idee",


    # neue university stopwords 
"übung",
"beispiel",
"studierende",
"grundlage",
"laborübung",
"entwicklung",
"funktion",
"praktikum",
"versuch",
"methode",
"modell",
"kompetente",
"teil",
"ziel",
"schulungsreihe",
"thema",
"teilnehmer",
"bereich",
"präsentation",
"einführung",
"uhr",
"termin",
"überblick",
"aspekt",
"kontakt",
"anmeldung",
"inhalt",
"kontakt",
"sprechstunde",
"anforderung",
"modell",
"aufbau",
"fahren",
"student",
"jeweils",
"studierenden",
"prof",
"seminar",
"seminare",
"–",
"fortlaufendes",
"schwerpunktmodul",
"entwickelt",
"handelns",
"ansatz",
"schwerpunktmoduls",
"einzubringen",
"semestern",
"semesternseminar",
"seminarnseminar",
"zulassung",
"rahmen",
"blick",
"bereit",
"teilnehmenden",
"gruppe",
"teilnehmerinnen",
"modul",
"ausdruck",
"einfluß",
"kolloquium",
"thematisiert",
"bereiche",
"vermittelt",
"anhand",
"schwerpunkt",
"seminars",
"lehrperson",
"deutsche",
"form",
"gruppen",
"gegenstand",
"anliegen",
"ansätzen",
"vorgestellt",
"ndie",
"forschungscolleges",
"erprobt",
"denkens",
"ansatzes",
"vorausgesetzt",
"montag",
"studium",
"blockseminar",
"veranstaltung",
"masterstudierende",
"datum",
"immatrikulierte",
"dienstagskolloquium",
"leistungsnachweise",
"leistungsnachweis",
"vgl",
"the",
"studiengangs",
"veranstaltung",
"seminar",
"vorlesung",
"kurs",
"modul",
"angebot",
"behandlung",
"themen",
"einführung",
"grundlagen",
"aufbau",
"überblick",
"inhalt",
"praxis",
"theorie",
"schwerpunkt",
"recht",
"rechts",
"rechtsgebiete",
"rechtswissenschaft",
"rechtsfragen",
"bereich",
"gebiete",
"gesetz",
"juristisch",
"aktuell",
"neue",
"verschiedenen",
"interdisziplinär",
"relevant",
"bedeutung",
"besonders",
"werden",
"sollen",
"kann",
"bietet",
"z.b.",
"u.a.",
"etc.",
"bspw.",
"ggf.",
"i.d.r.",
"family",
"relationship",
"practice",
"lesson",
"better",
"lawsuits",
"they",
"aid",
"united",
"register",
"between",
"language",
"about",
"einfache",
"right",
"acp",
"before",
"common",
"please",
"matrikel",
"registration",
"jurisdictions",
"across",
"limits",
"bundesverfassungsgerichts",
"künftigen",
"naturrecht",
"recent",
"schriftliches",
"literaturverarbeitung",
"kurzvortrag",
"genügt",
"seelmann",
"referats",
"lage",
"fighting",
"neuster",
"arbeitsmethoden",
"eugh",
"algorithmus",
"formel",
"adresse",
"resolution",
"dispute",
"module",
"literaturhinweise",
"civil",
"regulatory",
"menschenwürde",
"machthabern",


    # solariz stopwords 
    "ab",
  "aber",
  "abermaliges",
  "abermals",
  "abgerufen",
  "abgerufene",
  "abgerufener",
  "abgerufenes",
  "abgesehen",
  "acht",
  "aehnlich",
  "aehnliche",
  "aehnlichem",
  "aehnlichen",
  "aehnlicher",
  "aehnliches",
  "aehnlichste",
  "aehnlichstem",
  "aehnlichsten",
  "aehnlichster",
  "aehnlichstes",
  "aeusserst",
  "aeusserste",
  "aeusserstem",
  "aeussersten",
  "aeusserster",
  "aeusserstes",
  "ähnlich",
  "ähnliche",
  "ähnlichem",
  "ähnlichen",
  "ähnlicher",
  "ähnliches",
  "ähnlichst",
  "ähnlichste",
  "ähnlichstem",
  "ähnlichsten",
  "ähnlichster",
  "ähnlichstes",
  "alle",
  "allein",
  "alleine",
  "allem",
  "allemal",
  "allen",
  "allenfalls",
  "allenthalben",
  "aller",
  "allerdings",
  "allerlei",
  "alles",
  "allesamt",
  "allg",
  "allg.",
  "allgemein",
  "allgemeine",
  "allgemeinem",
  "allgemeinen",
  "allgemeiner",
  "allgemeines",
  "allgemeinste",
  "allgemeinstem",
  "allgemeinsten",
  "allgemeinster",
  "allgemeinstes",
  "allmählich",
  "allzeit",
  "allzu",
  "als",
  "alsbald",
  "also",
  "am",
  "an",
  "and",
  "andauernd",
  "andauernde",
  "andauerndem",
  "andauernden",
  "andauernder",
  "andauerndes",
  "ander",
  "andere",
  "anderem",
  "anderen",
  "anderenfalls",
  "anderer",
  "andererseits",
  "anderes",
  "anderm",
  "andern",
  "andernfalls",
  "anderr",
  "anders",
  "anderst",
  "anderweitig",
  "anderweitige",
  "anderweitigem",
  "anderweitigen",
  "anderweitiger",
  "anderweitiges",
  "anerkannt",
  "anerkannte",
  "anerkannter",
  "anerkanntes",
  "anfangen",
  "anfing",
  "angefangen",
  "angesetze",
  "angesetzt",
  "angesetzten",
  "angesetzter",
  "ans",
  "anscheinend",
  "ansetzen",
  "ansonst",
  "ansonsten",
  "anstatt",
  "anstelle",
  "arbeiten",
  "auch",
  "auf",
  "aufgehört",
  "aufgrund",
  "aufhören",
  "aufhörte",
  "aufzusuchen",
  "augenscheinlich",
  "augenscheinliche",
  "augenscheinlichem",
  "augenscheinlichen",
  "augenscheinlicher",
  "augenscheinliches",
  "augenscheinlichst",
  "augenscheinlichste",
  "augenscheinlichstem",
  "augenscheinlichsten",
  "augenscheinlichster",
  "augenscheinlichstes",
  "aus",
  "ausdrücken",
  "ausdrücklich",
  "ausdrückliche",
  "ausdrücklichem",
  "ausdrücklichen",
  "ausdrücklicher",
  "ausdrückliches",
  "ausdrückt",
  "ausdrückte",
  "ausgenommen",
  "ausgenommene",
  "ausgenommenem",
  "ausgenommenen",
  "ausgenommener",
  "ausgenommenes",
  "ausgerechnet",
  "ausgerechnete",
  "ausgerechnetem",
  "ausgerechneten",
  "ausgerechneter",
  "ausgerechnetes",
  "ausnahmslos",
  "ausnahmslose",
  "ausnahmslosem",
  "ausnahmslosen",
  "ausnahmsloser",
  "ausnahmsloses",
  "außen",
  "ausser",
  "ausserdem",
  "außerhalb",
  "äusserst",
  "äusserste",
  "äusserstem",
  "äussersten",
  "äusserster",
  "äusserstes",
  "author",
  "autor",
  "baelde",
  "bald",
  "bälde",
  "bearbeite",
  "bearbeiten",
  "bearbeitete",
  "bearbeiteten",
  "bedarf",
  "bedürfen",
  "bedurfte",
  "been",
  "befahl",
  "befiehlt",
  "befiehlte",
  "befohlene",
  "befohlens",
  "befragen",
  "befragte",
  "befragten",
  "befragter",
  "begann",
  "beginnen",
  "begonnen",
  "behalten",
  "behielt",
  "bei",
  "beide",
  "beidem",
  "beiden",
  "beider",
  "beiderlei",
  "beides",
  "beim",
  "beinahe",
  "beisammen",
  "beispielsweise",
  "beitragen",
  "beitrugen",
  "bekannt",
  "bekannte",
  "bekannter",
  "bekanntlich",
  "bekanntliche",
  "bekanntlichem",
  "bekanntlichen",
  "bekanntlicher",
  "bekanntliches",
  "bekennen",
  "benutzt",
  "bereits",
  "berichten",
  "berichtet",
  "berichtete",
  "berichteten",
  "besonders",
  "besser",
  "bessere",
  "besserem",
  "besseren",
  "besserer",
  "besseres",
  "bestehen",
  "besteht",
  "bestenfalls",
  "bestimmt",
  "bestimmte",
  "bestimmtem",
  "bestimmten",
  "bestimmter",
  "bestimmtes",
  "beträchtlich",
  "beträchtliche",
  "beträchtlichem",
  "beträchtlichen",
  "beträchtlicher",
  "beträchtliches",
  "betraechtlich",
  "betraechtliche",
  "betraechtlichem",
  "betraechtlichen",
  "betraechtlicher",
  "betraechtliches",
  "betreffend",
  "betreffende",
  "betreffendem",
  "betreffenden",
  "betreffender",
  "betreffendes",
  "bevor",
  "bez",
  "bez.",
  "bezgl",
  "bezgl.",
  "bezueglich",
  "bezüglich",
  "bietet",
  "bin",
  "bis",
  "bisher",
  "bisherige",
  "bisherigem",
  "bisherigen",
  "bisheriger",
  "bisheriges",
  "bislang",
  "bisschen",
  "bist",
  "bitte",
  "bleiben",
  "bleibt",
  "blieb",
  "bloss",
  "böden",
  "boeden",
  "brachte",
  "brachten",
  "brauchen",
  "braucht",
  "bräuchte",
  "bringen",
  "bsp",
  "bsp.",
  "bspw",
  "bspw.",
  "bzw",
  "bzw.",
  "ca",
  "ca.",
  "circa",
  "da",
  "dabei",
  "dadurch",
  "dafuer",
  "dafür",
  "dagegen",
  "daher",
  "dahin",
  "dahingehend",
  "dahingehende",
  "dahingehendem",
  "dahingehenden",
  "dahingehender",
  "dahingehendes",
  "dahinter",
  "damalige",
  "damaligem",
  "damaligen",
  "damaliger",
  "damaliges",
  "damals",
  "damit",
  "danach",
  "daneben",
  "dank",
  "danke",
  "danken",
  "dann",
  "dannen",
  "daran",
  "darauf",
  "daraus",
  "darf",
  "darfst",
  "darin",
  "darüber",
  "darüberhinaus",
  "darueber",
  "darueberhinaus",
  "darum",
  "darunter",
  "das",
  "daß",
  "dass",
  "dasselbe",
  "Dat",
  "davon",
  "davor",
  "dazu",
  "dazwischen",
  "dein",
  "deine",
  "deinem",
  "deinen",
  "deiner",
  "deines",
  "dem",
  "demgegenüber",
  "demgegenueber",
  "demgemaess",
  "demgemäss",
  "demnach",
  "demselben",
  "den",
  "denen",
  "denkbar",
  "denkbare",
  "denkbarem",
  "denkbaren",
  "denkbarer",
  "denkbares",
  "denn",
  "dennoch",
  "denselben",
  "der",
  "derart",
  "derartig",
  "derartige",
  "derartigem",
  "derartigen",
  "derartiger",
  "derem",
  "deren",
  "derer",
  "derjenige",
  "derjenigen",
  "derselbe",
  "derselben",
  "derzeit",
  "derzeitig",
  "derzeitige",
  "derzeitigem",
  "derzeitigen",
  "derzeitiges",
  "des",
  "deshalb",
  "desselben",
  "dessen",
  "dessenungeachtet",
  "desto",
  "desungeachtet",
  "deswegen",
  "dich",
  "die",
  "diejenige",
  "diejenigen",
  "dies",
  "diese",
  "dieselbe",
  "dieselben",
  "diesem",
  "diesen",
  "dieser",
  "dieses",
  "diesseitig",
  "diesseitige",
  "diesseitigem",
  "diesseitigen",
  "diesseitiger",
  "diesseitiges",
  "diesseits",
  "dinge",
  "dir",
  "direkt",
  "direkte",
  "direkten",
  "direkter",
  "doch",
  "doppelt",
  "dort",
  "dorther",
  "dorthin",
  "dran",
  "drauf",
  "drei",
  "dreißig",
  "drin",
  "dritte",
  "drüber",
  "drueber",
  "drum",
  "drunter",
  "du",
  "duerfte",
  "duerften",
  "duerftest",
  "duerftet",
  "dunklen",
  "durch",
  "durchaus",
  "durchweg",
  "durchwegs",
  "dürfen",
  "durfte",
  "dürfte",
  "durften",
  "dürften",
  "durftest",
  "dürftest",
  "durftet",
  "dürftet",
  "eben",
  "ebenfalls",
  "ebenso",
  "ect",
  "ect.",
  "ehe",
  "eher",
  "eheste",
  "ehestem",
  "ehesten",
  "ehester",
  "ehestes",
  "eigen",
  "eigene",
  "eigenem",
  "eigenen",
  "eigener",
  "eigenes",
  "eigenst",
  "eigentlich",
  "eigentliche",
  "eigentlichem",
  "eigentlichen",
  "eigentlicher",
  "eigentliches",
  "ein",
  "einbaün",
  "eine",
  "einem",
  "einen",
  "einer",
  "einerlei",
  "einerseits",
  "eines",
  "einfach",
  "einführen",
  "einführte",
  "einführten",
  "eingesetzt",
  "einig",
  "einige",
  "einigem",
  "einigen",
  "einiger",
  "einigermaßen",
  "einiges",
  "einmal",
  "einmalig",
  "einmalige",
  "einmaligem",
  "einmaligen",
  "einmaliger",
  "einmaliges",
  "eins",
  "einseitig",
  "einseitige",
  "einseitigen",
  "einseitiger",
  "einst",
  "einstmals",
  "einzig",
  "empfunden",
  "ende",
  "entgegen",
  "entlang",
  "entsprechend",
  "entsprechende",
  "entsprechendem",
  "entsprechenden",
  "entsprechender",
  "entsprechendes",
  "entweder",
  "er",
  "ergänze",
  "ergänzen",
  "ergänzte",
  "ergänzten",
  "ergo",
  "erhält",
  "erhalten",
  "erhielt",
  "erhielten",
  "erneut",
  "eröffne",
  "eröffnen",
  "eröffnet",
  "eröffnete",
  "eröffnetes",
  "erscheinen",
  "erst",
  "erste",
  "erstem",
  "ersten",
  "erster",
  "erstere",
  "ersterem",
  "ersteren",
  "ersterer",
  "ersteres",
  "erstes",
  "es",
  "etc",
  "etc.",
  "etliche",
  "etlichem",
  "etlichen",
  "etlicher",
  "etliches",
  "etwa",
  "etwaige",
  "etwas",
  "euch",
  "euer",
  "eure",
  "eurem",
  "euren",
  "eurer",
  "eures",
  "euretwegen",
  "fall",
  "falls",
  "fand",
  "fast",
  "ferner",
  "fertig",
  "finde",
  "finden",
  "findest",
  "findet",
  "folgend",
  "folgende",
  "folgendem",
  "folgenden",
  "folgender",
  "folgendermassen",
  "folgendes",
  "folglich",
  "for",
  "fordern",
  "fordert",
  "forderte",
  "forderten",
  "fort",
  "fortsetzen",
  "fortsetzt",
  "fortsetzte",
  "fortsetzten",
  "fragte",
  "frau",
  "frei",
  "freie",
  "freier",
  "freies",
  "fuer",
  "fuers",
  "fünf",
  "für",
  "fürs",
  "gab",
  "gaenzlich",
  "gaenzliche",
  "gaenzlichem",
  "gaenzlichen",
  "gaenzlicher",
  "gaenzliches",
  "gängig",
  "gängige",
  "gängigen",
  "gängiger",
  "gängiges",
  "ganz",
  "ganze",
  "ganzem",
  "ganzen",
  "ganzer",
  "ganzes",
  "gänzlich",
  "gänzliche",
  "gänzlichem",
  "gänzlichen",
  "gänzlicher",
  "gänzliches",
  "gar",
  "gbr",
  "geb",
  "geben",
  "geblieben",
  "gebracht",
  "gedurft",
  "geehrt",
  "geehrte",
  "geehrten",
  "geehrter",
  "gefallen",
  "gefälligst",
  "gefällt",
  "gefiel",
  "gegeben",
  "gegen",
  "gegenüber",
  "gegenueber",
  "gehabt",
  "gehalten",
  "gehen",
  "geht",
  "gekommen",
  "gekonnt",
  "gemacht",
  "gemaess",
  "gemäss",
  "gemeinhin",
  "gemocht",
  "genau",
  "genommen",
  "genug",
  "gepriesener",
  "gepriesenes",
  "gerade",
  "gern",
  "gesagt",
  "gesehen",
  "gestern",
  "gestrige",
  "getan",
  "geteilt",
  "geteilte",
  "getragen",
  "getrennt",
  "gewesen",
  "gewiss",
  "gewisse",
  "gewissem",
  "gewissen",
  "gewisser",
  "gewissermaßen",
  "gewisses",
  "gewollt",
  "geworden",
  "ggf",
  "ggf.",
  "gib",
  "gibt",
  "gilt",
  "gleich",
  "gleiche",
  "gleichem",
  "gleichen",
  "gleicher",
  "gleiches",
  "gleichsam",
  "gleichste",
  "gleichstem",
  "gleichsten",
  "gleichster",
  "gleichstes",
  "gleichwohl",
  "gleichzeitig",
  "gleichzeitige",
  "gleichzeitigem",
  "gleichzeitigen",
  "gleichzeitiger",
  "gleichzeitiges",
  "glücklicherweise",
  "gluecklicherweise",
  "gmbh",
  "gottseidank",
  "gratulieren",
  "gratuliert",
  "gratulierte",
  "groesstenteils",
  "grösstenteils",
  "gruendlich",
  "gründlich",
  "gut",
  "gute",
  "guten",
  "hab",
  "habe",
  "haben",
  "habt",
  "haette",
  "haeufig",
  "haeufige",
  "haeufigem",
  "haeufigen",
  "haeufiger",
  "haeufigere",
  "haeufigeren",
  "haeufigerer",
  "haeufigeres",
  "halb",
  "hallo",
  "halten",
  "hast",
  "hat",
  "hätt",
  "hatte",
  "hätte",
  "hatten",
  "hätten",
  "hattest",
  "hattet",
  "häufig",
  "häufige",
  "häufigem",
  "häufigen",
  "häufiger",
  "häufigere",
  "häufigeren",
  "häufigerer",
  "häufigeres",
  "hen",
  "her",
  "heraus",
  "herein",
  "herum",
  "heute",
  "heutige",
  "heutigem",
  "heutigen",
  "heutiger",
  "heutiges",
  "hier",
  "hierbei",
  "hiermit",
  "hiesige",
  "hiesigem",
  "hiesigen",
  "hiesiger",
  "hiesiges",
  "hin",
  "hindurch",
  "hinein",
  "hingegen",
  "hinlanglich",
  "hinlänglich",
  "hinten",
  "hintendran",
  "hinter",
  "hinterher",
  "hinterm",
  "hintern",
  "hinunter",
  "hoch",
  "höchst",
  "höchstens",
  "http",
  "hundert",
  "ich",
  "igitt",
  "ihm",
  "ihn",
  "ihnen",
  "ihr",
  "ihre",
  "ihrem",
  "ihren",
  "ihrer",
  "ihres",
  "ihretwegen",
  "ihrige",
  "ihrigen",
  "ihriges",
  "im",
  "immer",
  "immerhin",
  "immerwaehrend",
  "immerwaehrende",
  "immerwaehrendem",
  "immerwaehrenden",
  "immerwaehrender",
  "immerwaehrendes",
  "immerwährend",
  "immerwährende",
  "immerwährendem",
  "immerwährenden",
  "immerwährender",
  "immerwährendes",
  "immerzu",
  "important",
  "in",
  "indem",
  "indessen",
  "Inf.",
  "info",
  "infolge",
  "infolgedessen",
  "information",
  "innen",
  "innerhalb",
  "innerlich",
  "ins",
  "insbesondere",
  "insgeheim",
  "insgeheime",
  "insgeheimer",
  "insgesamt",
  "insgesamte",
  "insgesamter",
  "insofern",
  "inzwischen",
  "irgend",
  "irgendein",
  "irgendeine",
  "irgendeinem",
  "irgendeiner",
  "irgendeines",
  "irgendetwas",
  "irgendjemand",
  "irgendjemandem",
  "irgendwann",
  "irgendwas",
  "irgendwelche",
  "irgendwen",
  "irgendwenn",
  "irgendwer",
  "irgendwie",
  "irgendwo",
  "irgendwohin",
  "ist",
  "ja",
  "jaehrig",
  "jaehrige",
  "jaehrigem",
  "jaehrigen",
  "jaehriger",
  "jaehriges",
  "jährig",
  "jährige",
  "jährigem",
  "jährigen",
  "jähriges",
  "je",
  "jede",
  "jedem",
  "jeden",
  "jedenfalls",
  "jeder",
  "jederlei",
  "jedes",
  "jedesmal",
  "jedoch",
  "jeglichem",
  "jeglichen",
  "jeglicher",
  "jegliches",
  "jemals",
  "jemand",
  "jemandem",
  "jemanden",
  "jemandes",
  "jene",
  "jenem",
  "jenen",
  "jener",
  "jenes",
  "jenseitig",
  "jenseitigem",
  "jenseitiger",
  "jenseits",
  "jetzt",
  "jung",
  "junge",
  "jungem",
  "jungen",
  "junger",
  "junges",
  "kaeumlich",
  "kam",
  "kann",
  "kannst",
  "kaum",
  "käumlich",
  "kein",
  "keine",
  "keinem",
  "keinen",
  "keiner",
  "keinerlei",
  "keines",
  "keineswegs",
  "klar",
  "klare",
  "klaren",
  "klares",
  "klein",
  "kleinen",
  "kleiner",
  "kleines",
  "koennen",
  "koennt",
  "koennte",
  "koennten",
  "koenntest",
  "koenntet",
  "komme",
  "kommen",
  "kommt",
  "konkret",
  "konkrete",
  "konkreten",
  "konkreter",
  "konkretes",
  "könn",
  "können",
  "könnt",
  "konnte",
  "könnte",
  "konnten",
  "könnten",
  "konntest",
  "könntest",
  "konntet",
  "könntet",
  "kuenftig",
  "kuerzlich",
  "kuerzlichst",
  "künftig",
  "kürzlich",
  "kürzlichst",
  "laengst",
  "lag",
  "lagen",
  "langsam",
  "längst",
  "längstens",
  "lassen",
  "laut",
  "lediglich",
  "leer",
  "legen",
  "legte",
  "legten",
  "leicht",
  "leider",
  "lesen",
  "letze",
  "letzte",
  "letzten",
  "letztendlich",
  "letztens",
  "letztere",
  "letzterem",
  "letzterer",
  "letzteres",
  "letztes",
  "letztlich",
  "lichten",
  "liegt",
  "liest",
  "links",
  "mache",
  "machen",
  "machst",
  "macht",
  "machte",
  "machten",
  "mag",
  "magst",
  "mal",
  "man",
  "manch",
  "manche",
  "manchem",
  "manchen",
  "mancher",
  "mancherlei",
  "mancherorts",
  "manches",
  "manchmal",
  "mann",
  "margin",
  "massgebend",
  "massgebende",
  "massgebendem",
  "massgebenden",
  "massgebender",
  "massgebendes",
  "massgeblich",
  "massgebliche",
  "massgeblichem",
  "massgeblichen",
  "massgeblicher",
  "mehr",
  "mehrere",
  "mehrerer",
  "mehrfach",
  "mehrmalig",
  "mehrmaligem",
  "mehrmaliger",
  "mehrmaliges",
  "mein",
  "meine",
  "meinem",
  "meinen",
  "meiner",
  "meines",
  "meinetwegen",
  "meins",
  "meist",
  "meiste",
  "meisten",
  "meistens",
  "meistenteils",
  "meta",
  "mich",
  "mindestens",
  "mir",
  "mit",
  "miteinander",
  "mitgleich",
  "mithin",
  "mitnichten",
  "mittels",
  "mittelst",
  "mitten",
  "mittig",
  "mitunter",
  "mitwohl",
  "mochte",
  "möchte",
  "möchten",
  "möchtest",
  "moechte",
  "moeglich",
  "moeglichst",
  "moeglichste",
  "moeglichstem",
  "moeglichsten",
  "moeglichster",
  "mögen",
  "möglich",
  "mögliche",
  "möglichen",
  "möglicher",
  "möglicherweise",
  "möglichst",
  "möglichste",
  "möglichstem",
  "möglichsten",
  "möglichster",
  "morgen",
  "morgige",
  "muessen",
  "muesst",
  "muesste",
  "muss",
  "müssen",
  "musst",
  "müßt",
  "musste",
  "müsste",
  "mussten",
  "müssten",
  "nach",
  "nachdem",
  "nacher",
  "nachher",
  "nachhinein",
  "nächste",
  "nacht",
  "naechste",
  "naemlich",
  "nahm",
  "nämlich",
  "naturgemaess",
  "naturgemäss",
  "natürlich",
  "ncht",
  "neben",
  "nebenan",
  "nehmen",
  "nein",
  "neu",
  "neue",
  "neuem",
  "neuen",
  "neuer",
  "neuerdings",
  "neuerlich",
  "neuerliche",
  "neuerlichem",
  "neuerlicher",
  "neuerliches",
  "neues",
  "neulich",
  "neun",
  "nicht",
  "nichts",
  "nichtsdestotrotz",
  "nichtsdestoweniger",
  "nie",
  "niemals",
  "niemand",
  "niemandem",
  "niemanden",
  "niemandes",
  "nimm",
  "nimmer",
  "nimmt",
  "nirgends",
  "nirgendwo",
  "noch",
  "noetigenfalls",
  "nötigenfalls",
  "nun",
  "nur",
  "nutzen",
  "nutzt",
  "nützt",
  "nutzung",
  "ob",
  "oben",
  "ober",
  "oberen",
  "oberer",
  "oberhalb",
  "oberste",
  "obersten",
  "oberster",
  "obgleich",
  "obs",
  "obschon",
  "obwohl",
  "oder",
  "oefter",
  "oefters",
  "off",
  "offenkundig",
  "offenkundige",
  "offenkundigem",
  "offenkundigen",
  "offenkundiger",
  "offenkundiges",
  "offensichtlich",
  "offensichtliche",
  "offensichtlichem",
  "offensichtlichen",
  "offensichtlicher",
  "offensichtliches",
  "oft",
  "öfter",
  "öfters",
  "oftmals",
  "ohne",
  "ohnedies",
  "online",
  "paar",
  "partout",
  "per",
  "persoenlich",
  "persoenliche",
  "persoenlichem",
  "persoenlicher",
  "persoenliches",
  "persönlich",
  "persönliche",
  "persönlicher",
  "persönliches",
  "pfui",
  "ploetzlich",
  "ploetzliche",
  "ploetzlichem",
  "ploetzlicher",
  "ploetzliches",
  "plötzlich",
  "plötzliche",
  "plötzlichem",
  "plötzlicher",
  "plötzliches",
  "pro",
  "quasi",
  "reagiere",
  "reagieren",
  "reagiert",
  "reagierte",
  "recht",
  "rechts",
  "regelmäßig",
  "reichlich",
  "reichliche",
  "reichlichem",
  "reichlichen",
  "reichlicher",
  "restlos",
  "restlose",
  "restlosem",
  "restlosen",
  "restloser",
  "restloses",
  "richtiggehend",
  "richtiggehende",
  "richtiggehendem",
  "richtiggehenden",
  "richtiggehender",
  "richtiggehendes",
  "rief",
  "rund",
  "rundheraus",
  "rundum",
  "runter",
  "sage",
  "sagen",
  "sagt",
  "sagte",
  "sagten",
  "sagtest",
  "sagtet",
  "samt",
  "sämtliche",
  "sang",
  "sangen",
  "sattsam",
  "schätzen",
  "schätzt",
  "schätzte",
  "schätzten",
  "scheinbar",
  "scheinen",
  "schlechter",
  "schlicht",
  "schlichtweg",
  "schließlich",
  "schlussendlich",
  "schnell",
  "schon",
  "schreibe",
  "schreiben",
  "schreibens",
  "schreiber",
  "schwerlich",
  "schwerliche",
  "schwerlichem",
  "schwerlichen",
  "schwerlicher",
  "schwerliches",
  "schwierig",
  "sechs",
  "sect",
  "sehe",
  "sehen",
  "sehr",
  "sehrwohl",
  "seht",
  "sei",
  "seid",
  "seien",
  "seiest",
  "seiet",
  "sein",
  "seine",
  "seinem",
  "seinen",
  "seiner",
  "seines",
  "seit",
  "seitdem",
  "seite",
  "seiten",
  "seither",
  "selbe",
  "selben",
  "selber",
  "selbst",
  "selbstredend",
  "selbstredende",
  "selbstredendem",
  "selbstredenden",
  "selbstredender",
  "selbstredendes",
  "seltsamerweise",
  "senke",
  "senken",
  "senkt",
  "senkte",
  "senkten",
  "setzen",
  "setzt",
  "setzte",
  "setzten",
  "sich",
  "sicher",
  "sicherlich",
  "sie",
  "sieben",
  "siebte",
  "siehe",
  "sieht",
  "sind",
  "singen",
  "singt",
  "so",
  "sobald",
  "sodaß",
  "soeben",
  "sofern",
  "sofort",
  "sog",
  "sogar",
  "sogleich",
  "solange",
  "solc",
  "solc hen",
  "solch",
  "solche",
  "solchem",
  "solchen",
  "solcher",
  "solches",
  "soll",
  "sollen",
  "sollst",
  "sollt",
  "sollte",
  "sollten",
  "solltest",
  "solltet",
  "somit",
  "sondern",
  "sonst",
  "sonstig",
  "sonstige",
  "sonstigem",
  "sonstiger",
  "sonstwo",
  "sooft",
  "soviel",
  "soweit",
  "sowie",
  "sowieso",
  "sowohl",
  "später",
  "spielen",
  "startet",
  "startete",
  "starteten",
  "statt",
  "stattdessen",
  "steht",
  "steige",
  "steigen",
  "steigt",
  "stellenweise",
  "stellenweisem",
  "stellenweisen",
  "stets",
  "stieg",
  "stiegen",
  "such",
  "suchen",
  "tages",
  "tat",
  "tät",
  "tatsächlich",
  "tatsächlichen",
  "tatsächlicher",
  "tatsächliches",
  "tatsaechlich",
  "tatsaechlichen",
  "tatsaechlicher",
  "tatsaechliches",
  "tausend",
  "teile",
  "teilen",
  "teilte",
  "teilten",
  "tief",
  "titel",
  "toll",
  "total",
  "trage",
  "tragen",
  "trägt",
  "trotzdem",
  "trug",
  "tun",
  "tust",
  "tut",
  "txt",
  "übel",
  "über",
  "überall",
  "überallhin",
  "überaus",
  "überdies",
  "überhaupt",
  "überll",
  "übermorgen",
  "üblicherweise",
  "übrig",
  "übrigens",
  "ueber",
  "ueberall",
  "ueberallhin",
  "ueberaus",
  "ueberdies",
  "ueberhaupt",
  "uebermorgen",
  "ueblicherweise",
  "uebrig",
  "uebrigens",
  "um",
  "ums",
  "umso",
  "umstaendehalber",
  "umständehalber",
  "unbedingt",
  "unbedingte",
  "unbedingter",
  "unbedingtes",
  "und",
  "unerhoert",
  "unerhoerte",
  "unerhoertem",
  "unerhoerten",
  "unerhoerter",
  "unerhoertes",
  "unerhört",
  "unerhörte",
  "unerhörtem",
  "unerhörten",
  "unerhörter",
  "unerhörtes",
  "ungefähr",
  "ungemein",
  "ungewoehnlich",
  "ungewoehnliche",
  "ungewoehnlichem",
  "ungewoehnlichen",
  "ungewoehnlicher",
  "ungewoehnliches",
  "ungewöhnlich",
  "ungewöhnliche",
  "ungewöhnlichem",
  "ungewöhnlichen",
  "ungewöhnlicher",
  "ungewöhnliches",
  "ungleich",
  "ungleiche",
  "ungleichem",
  "ungleichen",
  "ungleicher",
  "ungleiches",
  "unmassgeblich",
  "unmassgebliche",
  "unmassgeblichem",
  "unmassgeblichen",
  "unmassgeblicher",
  "unmassgebliches",
  "unmoeglich",
  "unmoegliche",
  "unmoeglichem",
  "unmoeglichen",
  "unmoeglicher",
  "unmoegliches",
  "unmöglich",
  "unmögliche",
  "unmöglichen",
  "unmöglicher",
  "unnötig",
  "uns",
  "unsaeglich",
  "unsaegliche",
  "unsaeglichem",
  "unsaeglichen",
  "unsaeglicher",
  "unsaegliches",
  "unsagbar",
  "unsagbare",
  "unsagbarem",
  "unsagbaren",
  "unsagbarer",
  "unsagbares",
  "unsäglich",
  "unsägliche",
  "unsäglichem",
  "unsäglichen",
  "unsäglicher",
  "unsägliches",
  "unse",
  "unsem",
  "unsen",
  "unser",
  "unsere",
  "unserem",
  "unseren",
  "unserer",
  "unseres",
  "unserm",
  "unses",
  "unsre",
  "unsrem",
  "unsren",
  "unsrer",
  "unsres",
  "unstreitig",
  "unstreitige",
  "unstreitigem",
  "unstreitigen",
  "unstreitiger",
  "unstreitiges",
  "unten",
  "unter",
  "unterbrach",
  "unterbrechen",
  "untere",
  "unterem",
  "unteres",
  "unterhalb",
  "unterste",
  "unterster",
  "unterstes",
  "unwichtig",
  "unzweifelhaft",
  "unzweifelhafte",
  "unzweifelhaftem",
  "unzweifelhaften",
  "unzweifelhafter",
  "unzweifelhaftes",
  "usw",
  "usw.",
  "vergangen",
  "vergangene",
  "vergangener",
  "vergangenes",
  "vermag",
  "vermögen",
  "vermutlich",
  "vermutliche",
  "vermutlichem",
  "vermutlichen",
  "vermutlicher",
  "vermutliches",
  "veröffentlichen",
  "veröffentlicher",
  "veröffentlicht",
  "veröffentlichte",
  "veröffentlichten",
  "veröffentlichtes",
  "verrate",
  "verraten",
  "verriet",
  "verrieten",
  "version",
  "versorge",
  "versorgen",
  "versorgt",
  "versorgte",
  "versorgten",
  "versorgtes",
  "viel",
  "viele",
  "vielen",
  "vieler",
  "vielerlei",
  "vieles",
  "vielleicht",
  "vielmalig",
  "vielmals",
  "vier",
  "voellig",
  "voellige",
  "voelligem",
  "voelligen",
  "voelliger",
  "voelliges",
  "voelligst",
  "vollends",
  "völlig",
  "völlige",
  "völligem",
  "völligen",
  "völliger",
  "völliges",
  "völligst",
  "vollstaendig",
  "vollstaendige",
  "vollstaendigem",
  "vollstaendigen",
  "vollstaendiger",
  "vollstaendiges",
  "vollständig",
  "vollständige",
  "vollständigem",
  "vollständigen",
  "vollständiger",
  "vollständiges",
  "vom",
  "von",
  "vor",
  "voran",
  "vorbei",
  "vorgestern",
  "vorher",
  "vorherig",
  "vorherige",
  "vorherigem",
  "vorheriger",
  "vorne",
  "vorüber",
  "vorueber",
  "wachen",
  "waehrend",
  "waehrenddessen",
  "waere",
  "während",
  "währenddessen",
  "wann",
  "war",
  "wär",
  "wäre",
  "waren",
  "wären",
  "warst",
  "wart",
  "warum",
  "was",
  "weder",
  "weg",
  "wegen",
  "weil",
  "weiß",
  "weit",
  "weiter",
  "weitere",
  "weiterem",
  "weiteren",
  "weiterer",
  "weiteres",
  "weiterhin",
  "weitestgehend",
  "weitestgehende",
  "weitestgehendem",
  "weitestgehenden",
  "weitestgehender",
  "weitestgehendes",
  "weitgehend",
  "weitgehende",
  "weitgehendem",
  "weitgehenden",
  "weitgehender",
  "weitgehendes",
  "welche",
  "welchem",
  "welchen",
  "welcher",
  "welches",
  "wem",
  "wen",
  "wenig",
  "wenige",
  "weniger",
  "wenigstens",
  "wenn",
  "wenngleich",
  "wer",
  "werde",
  "werden",
  "werdet",
  "weshalb",
  "wessen",
  "weswegen",
  "wichtig",
  "wie",
  "wieder",
  "wiederum",
  "wieso",
  "wieviel",
  "wieviele",
  "wievieler",
  "wiewohl",
  "will",
  "willst",
  "wir",
  "wird",
  "wirklich",
  "wirklichem",
  "wirklicher",
  "wirkliches",
  "wirst",
  "wo",
  "wobei",
  "wodurch",
  "wofuer",
  "wofür",
  "wogegen",
  "woher",
  "wohin",
  "wohingegen",
  "wohl",
  "wohlgemerkt",
  "wohlweislich",
  "wolle",
  "wollen",
  "wollt",
  "wollte",
  "wollten",
  "wolltest",
  "wolltet",
  "womit",
  "womoeglich",
  "womoegliche",
  "womoeglichem",
  "womoeglichen",
  "womoeglicher",
  "womoegliches",
  "womöglich",
  "womögliche",
  "womöglichem",
  "womöglichen",
  "womöglicher",
  "womögliches",
  "woran",
  "woraufhin",
  "woraus",
  "worden",
  "worin",
  "wuerde",
  "wuerden",
  "wuerdest",
  "wuerdet",
  "wurde",
  "würde",
  "wurden",
  "würden",
  "wurdest",
  "würdest",
  "wurdet",
  "würdet",
  "www",
  "x",
  "z.B.",
  "zahlreich",
  "zahlreichem",
  "zahlreicher",
  "zB",
  "zb.",
  "zehn",
  "zeitweise",
  "zeitweisem",
  "zeitweisen",
  "zeitweiser",
  "ziehen",
  "zieht",
  "ziemlich",
  "ziemliche",
  "ziemlichem",
  "ziemlichen",
  "ziemlicher",
  "ziemliches",
  "zirka",
  "zog",
  "zogen",
  "zu",
  "zudem",
  "zuerst",
  "zufolge",
  "zugleich",
  "zuletzt",
  "zum",
  "zumal",
  "zumeist",
  "zumindest",
  "zunächst",
  "zunaechst",
  "zur",
  "zurück",
  "zurueck",
  "zusammen",
  "zusehends",
  "zuviel",
  "zuviele",
  "zuvieler",
  "zuweilen",
  "zwanzig",
  "zwar",
  "zwei",
  "zweifelsfrei",
  "zweifelsfreie",
  "zweifelsfreiem",
  "zweifelsfreien",
  "zweifelsfreier",
  "zweifelsfreies",
  "zwischen",
  "zwölf",

  # weitere stopwords 
  "et", "et al", "al",
  "exercise", 
  "projektpraktikum",
  "ii", "qe ii", "qe"
  # "strongly", "correlated", 
  # "englischen",
  # "mögliche",
  # "eingeführt",
  # "milestones",
  # "wesentlichen",
  # "zwischen",
  # "based",
  # "diskutiert",
  # "hergestellten", "einzelne",
  # "theoretischen einweisung",
  # "ökonomischen einsatzfeldern",
]

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)

## Model

Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

In [46]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# UMAP Settings
umap_model = UMAP(n_neighbors=6, min_dist=0.1, n_components=6, metric="cosine", random_state=13)

# HDBSCAN Settings
hdbscan_model = HDBSCAN(min_cluster_size=4, gen_min_span_tree=True, prediction_data=True)

# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

In [47]:
# BERTopic initialisieren
topic_model = BERTopic(
  embedding_model=embedding_model,
  #min_topic_size=10,
  #nr_topics=nr_topics, 
  language="multilingual",
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 30,
  representation_model=representation_model
)

Training

In [48]:
topic_model_quanten = topic_model.fit(docs)

Evaluation

In [49]:
# BERTopic auf Test-Daten anwenden
topics, probs = topic_model_quanten.transform(test_set)

In [50]:
topic_model_quanten.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,43,-1_quantum_quantum computing_codes_qubits,"[quantum, quantum computing, codes, qubits, qu...",[Proseminare Proseminar 1\r\r\nAngewandte Medi...
1,0,25,0_quantum_neutrinos_teilchenphysik_physics,"[quantum, neutrinos, teilchenphysik, physics, ...","[STS-MINT: STS in Science, Technology, Enginee..."
2,1,24,1_post quantum_kryptographie_cryptography_secure,"[post quantum, kryptographie, cryptography, se...",[Advanced Topics in Post-Quantum Cryptography ...
3,2,21,2_quantum technologies_photonic_photonic quant...,"[quantum technologies, photonic, photonic quan...",[Hauptseminar über aktuelle Themen der Photoni...
4,3,17,3_quantum_quantum materials_topological_scatte...,"[quantum, quantum materials, topological, scat...",[Quantenmechanik 2 1. Time dependent Hamiltoni...
5,4,16,4_quantum_magnetic_quantum system_quantenphysik,"[quantum, magnetic, quantum system, quantenphy...",[Quantensensorik * First date (Vorbesprechung)...
6,5,14,5_quantum sensors_nanostructures_nano amp_halb...,"[quantum sensors, nanostructures, nano amp, ha...",[Halbleitersensoren Einführend wird ein Überbl...
7,6,13,6_quantum_quantum computing_quantencomputer_qu...,"[quantum, quantum computing, quantencomputer, ...",[Quantum Computing The course provides an intr...
8,7,13,7_quantum_quantum computation_quantum computin...,"[quantum, quantum computation, quantum computi...",[Quantum Computing (Chalmers University) Eleme...
9,8,11,8_quantum_computing_quantum computing_quantum ...,"[quantum, computing, quantum computing, quantu...",[Seminar on Integration of Quantum Computing A...


In [51]:
# Outlier reduzieren
topics = topic_model_quanten.reduce_outliers(test_set, topics)

In [52]:
topics

[np.int64(6),
 np.int64(4),
 np.int64(7),
 np.int64(7),
 np.int64(6),
 np.int64(10),
 np.int64(6),
 np.int64(8),
 np.int64(2),
 np.int64(2),
 np.int64(1),
 np.int64(1),
 np.int64(4),
 np.int64(4),
 np.int64(2),
 np.int64(13),
 np.int64(4),
 np.int64(4),
 np.int64(3),
 np.int64(4),
 np.int64(2),
 np.int64(2),
 np.int64(7),
 np.int64(7),
 np.int64(4),
 np.int64(13),
 np.int64(10),
 np.int64(10),
 np.int64(5),
 np.int64(5),
 np.int64(8),
 np.int64(6),
 np.int64(3),
 np.int64(3),
 np.int64(8),
 np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(0),
 np.int64(0),
 np.int64(6),
 np.int64(1),
 np.int64(12),
 np.int64(1),
 np.int64(1),
 np.int64(3),
 np.int64(2),
 np.int64(11),
 np.int64(7),
 np.int64(10)]

In [53]:
# Resultierende Topic-Nummern mit den Representations (= relevante Begriffe) zu einem Datensatz kombinieren
dataframe_with_results_left = pd.DataFrame(topics, columns = ["Topic"])
dataframe_with_results_right = pd.DataFrame(topic_model_quanten.get_topic_info().set_index('Topic')[['Representation']])
dataframe_with_results = dataframe_with_results_left.join(dataframe_with_results_right, on="Topic")

In [55]:
row_number = 0
metric = 0
while row_number < len(ground_truth):
  # Den Goldstandard in eine Liste von Keywords umwandeln
  ground_truth_current_iteration = ground_truth[row_number].split(", ")
  result_current_iteration = dataframe_with_results.at[row_number, "Representation"]
  # Überprüfen, ob irgendein Begriff aus dem Resultat im Goldstandard zum Text vorkommt (1 = ja, 0 = nein)
  if any(element in result_current_iteration for element in ground_truth_current_iteration):
      metric += 1
  else: 
      metric += 0
      print(result_current_iteration)
      print(ground_truth_current_iteration)
      print("--------------------------------------------------------------------------------------------------")
  row_number = row_number+1

metric_score = metric/row_number
print(metric_score)

['quantum', 'magnetic', 'quantum system', 'quantenphysik', 'magnetic resonance', 'qubits', 'particles', 'quantum technologies', 'quantum systems', 'quantenmechanik']
['quantum technology', 'technology', 'quantum mechanics', 'mechanics', 'quantum physics', 'physics', 'entanglement', 'information']
--------------------------------------------------------------------------------------------------
['quantum', 'quantum computation', 'quantum computing', 'quantum fourier', 'quantum circuits', 'quantum fourier transform', 'algorithm quantum', 'quantum error correction', 'quantum error', 's algorithm']
['quantencomputing', 'quantenmechanik', 'computing', ' quantengatter', 'gatter', 'algorithmus', 'quantenschlüsselverteilung', 'quantum key distribution', 'shor', 'grover', 'quantenfehlerkorrektur', 'fehlerkorrektur', 'verschränkung', 'quantencomputer', 'computer', 'nichtlokalität']
--------------------------------------------------------------------------------------------------
['quantum', 'sup